# HSE 2023: Введение в машинное обучение БИ 22/23

## Домашнее задание № 3


# Внимание!

* Некоторые задания требуют значительного времени для выполнения (особенно часть с лемматизацией), поэтому **лучше приступить к выполнению домашнего задания как можно раньше** 

* Решения обязательно должны содержать комментарии, все полученные результаты должны сопровождаться выводами (для этого удобно использовать ячейки markdown)

In [1]:
from typing import Tuple, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

sns.set(style="darkgrid")

## ЧАСТЬ 1: Логистическая регрессия

Будем решать задачу бинарной классификации с помошью логистической регрессии.
Для борьбы с переобучением будем использовать регуляризацию - комбинацию $L_1$ и $L_2$ (Elastic Net loss).

Для каждого объекта обучающей выборки, заданного своим признаковым описанием $x_i\in\mathbb{R}^{K}$ (вектор из $k$ признаков), указан его класс $y_i$ (одно из двух значений). Параметрами модели являются смещение $w_0\in\mathbb{R}$ и веса $w\in\mathbb{R}^K$.

Таким образом, оптимизируемый функционал (Elastic Net loss) можно записать в виде:

$$L(w, w_0) = \frac{1}{N} \sum_{i=1}^N \ln(1+\exp(-y_i(w^\top x_i+w_0))) + \gamma \|w\|_1 + \beta \|w\|_2^2$$.

#### 1. [0.5 балла] Запишите формулу градиента Elastic Net loss (с выводом этой формулы, для форматирования лучше использовать Latex)

##### Put your markdown formulas here

#### 2. [0.25 балла] Реализуйте функцию вычисления Elastic Net loss

In [ ]:
def loss(X, y, w: List[float], w0: float, gamma=1., beta=1.) -> float:
    return # your code here

#### 3. [0.25 балла] Реализуйте функцию вычисления градиента

In [ ]:
def get_grad(X, y, w: List[float], w0: float, gamma=1., beta=1.) -> Tuple[List[float], float]:
    grad_w0 = # your code here
    grad_w = # your code here
    
    return grad_w, grad_w0

#### Проверьте корректность реализованных функций

In [ ]:
np.random.seed(42)
X = np.random.multivariate_normal(np.arange(5), np.eye(5), size=10)
y = np.random.binomial(1, 0.42, size=10)
w, w0 = np.random.normal(size=5), np.random.normal()

grad_w, grad_w0 = get_grad(X, y, w, w0)
assert(np.allclose(grad_w,
                   [-2.73262076, -1.87176281, 1.30051144, 2.53598941, -2.71198109],
                   rtol=1e-2) & \
       np.allclose(grad_w0,
                   -0.2078231418067844, 
                   rtol=1e-2)
)

####  4. [1 балл]  Реализуйте алгоритм градиентного спуска, поддержав различные критерии остановки: ограничение на количество итераций (max_iter) и ограничение на размер шага ([tolerance](https://nl.mathworks.com/help/optim/ug/tolerances-and-stopping-criteria.html)).

Проверьте корректность реализации, визуализировав полученную разделяющую поверхность ([plot_decision_boundary](https://towardsdatascience.com/logistic-regression-from-scratch-in-python-ec66603592e2)).

Ниже представлен шаблон класса, соответствующий стандартному API моделей sklearn.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
class Logit(BaseEstimator, ClassifierMixin):
    def __init__(self, beta=1.0, gamma=1.0, lr=1e-3, tolerance=0.01, max_iter=1000, random_state=42):  
        self.beta = beta        
        self.gamma = gamma
        self.tolerance= tolerance
        self.max_iter= max_iter
        self.learning_rate = lr
        self.random_state = random_state
        # you may additional properties if you wish
        
    def fit(self, X, y):
        # add weights and bias and optimize Elastic Net loss over (X,y) dataset
        # save history of optimization steps
        
        # your code here

        return self
    
    def predict(self, X):
        # return vector of predicted labels for each object from X
        # your code here

        return predict
        
    def predict_proba(self, X):
        return np.array([1 / (1 + np.exp(np.dot(X, self.w) + self.w0)),\
                         1 / (1 + np.exp(-np.dot(X, self.w) - self.w0))])

In [ ]:
# sample data to test your model
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=180, n_features=2, n_redundant=0, n_informative=2,
                               random_state=42, n_clusters_per_class=1)

In [ ]:
# a function to plot the decision boundary
def plot_decision_boundary(model, X, y):
    fig = plt.figure()
    X1min, X2min = X.min(axis=0)
    X1max, X2max = X.max(axis=0)
    x1, x2 = np.meshgrid(np.linspace(X1min, X1max, 200),
                         np.linspace(X2min, X2max, 200))
    ypred = model.predict(np.c_[x1.ravel(), x2.ravel()])
    ypred = ypred.reshape(x1.shape)
    
    plt.contourf(x1, x2, ypred, alpha=.4)
    plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
model = Logit(0,0)
y[y == 0] = -1
model.fit(X, y)
plot_decision_boundary(model, X, y)

#### 5. [0.25 балла] Постройте график зависимости значения функции потерь от номера итерации градиентного спуска.

In [ ]:
# your code here

## Часть 2: Support Vector Machines

#### 6. [2 балла] Обучите [SVM](https://scikit-learn.org/stable/modules/svm.html) классификатор на том же наборе данных.

Поисследуйте влияние гиперпараметров на качество обученной модели:
+ Попробуйте несколько ядер: линейное, полиномиальное, RBF (и другие, если хотите). У некоторых ядер есть гиперметры: не забудьте поэкспериментировать с ними!
+ Попробуйте разные коэффициенты регуляризации

В качестве метрик качества будем использовать accuracy, roc_auc и f1 score. 
Постройте графики зависимости метрик качества от гиперпараметров.

Какие выводы можно сделать на основе проведенных экспериментов? Насколько чувствительны ядра к гиперпараметрам? Какое ядро склонно приводить к переобучению? Насколько сильно на качество моделей влияет регуляризация?

In [ ]:
# your code here

## Часть 3: Natural Language Processing

#### 7. [1.75 балла] Подготовка данных

Подготовим набор данных, который будем использовать для бинарной и многоклассовой классификации.

0. Выберите **шесть** любимых писателей-прозаиков (укажите, кого вы выбрали) и скачайте  <a href="https://www.kaggle.com/d0rj3228/russian-literature?select=prose">данные</a> из раздела **проза**
1. Подготовьте собственный датасет из выбранных авторов: 
    * разделите каждый текст на предложения так, чтобы представить данные в виде *предложение* and *автор* (каждая строка обучающего набора данных содержит ровно одно предложение и одного автора текста, откуда было взято это предложение);
    * удалите короткие предложения (считаем, что предложение короткое, если в нем меньше 15 символов);
    * зафиксируйте random state и случайно сформируйте выборку из предложений размера "5k : 15k : 8k : 11k : 20k : 3k" в разбивке по авторам соответственно;
    
    Пример полученных данных:
    
    <center> 
    <table>
        <tr>
            <th> sentence </th>
            <th> author </th>
        </tr> 
        <tr><td> Несколько лет тому назад в одном из своих поместий жил старинный русской барин, Кирила Петрович Троекуров. </td><td> Пушкин </td><td> 
        <tr><td> Уже более недели приезжий господин жил в городе, разъезжая по вечеринкам и обедам и таким образом проводя, как говорится, очень приятно время. </td><td> Гоголь </td><td> 
        <tr><td> ... </td><td> ... </td><td> 
        <tr><td> Я жил недорослем, гоняя голубей и играя в чехарду с дворовыми мальчишками. </td><td> Пушкин </td><td>         
    </table>
</center>
     
2. Предварительная обработка данных: 
    * токенизируйте предложения, удалите все стоп-слова (nltk.corpus.stopwords), знаки пунктуации (string.punctuation) и числа;
    * преобразуйте все символы в нижний регистр и примените стемминг или лемматизацию (на свое усмотрение)
    * постройте векторные представления предложений с помощью **bag of words** и **tf-idf** (используйте средства sklearn)
    * обратите внимание на разницу между полученными векторными представлениями: чем отличаются векторы, полученные с помощью **bag of words** и **tf-idf**?

In [ ]:
# your code here

###  Бинарная классификация

#### 8. [2 балла] Обучете логистическую регрессию (написанную вами реализацию) и SVC (реализацию SVM из sklearn) 

* выберите **двух** любых авторов из сформированного в задании 7 набора данных
* проверьте, сбалансированны ли классы
* разделите данные на тренировочную и тестовую выборки, выделив под test 0.3 всех предложений (не забудьте зафиксировать random state)
* с помощью GridSearchCV найдите оптимальные гиперпараметры  моделей (согласно метрике F1 score) и используйте найденные гиперпараметры при выполнении следующих пунктов задания
* постройте графики зависимости F1 score от гиперпараметров
* постройте confusion matrix для train и test
* вычислите другие метрики качества для обученной модели (удобно воспользоваться реализацией из sklearn) 
* проанализируйте полученные значения метрик, сделайте вывод о качестве обученной модели


In [ ]:
# your code here

#### 9. [1 балл] ROC AUC

Можно контролировать статистические ошибки первого и второго типов, используя различные пороговые значения для определения классов. Постройте ROC-кривые для логистической регрессии и SVC. Подберите такой порог, чтобы частота ложноположительных срабатываний составляла не более 30%. Обратите внимание на параметр `thresholds` в sklearn roc_curve.

In [ ]:
# your code here

### Многоклассовая классификация

#### 10. [1 балл] Примените <a href="https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsOneClassifier.html">OneVsOneClassifier</a> из sklearn к реализованной логистической регресии для того, чтобы получить многоклассовый линейный классификатор:
* используйте набор данных, подготовленный в задании 7
* разделите данные на тренировочную и тестовую выборки, выделив под test 0.3 всех предложений (не забудьте зафиксировать random state)
* с помощью GridSearchCV найдите оптимальные гиперпараметры моделей (согласно метрике F1 score)
* постройте confusion matrix для train и test
* вычислите другие метрики качества для обученной модели (удобно воспользоваться реализацией из sklearn)
* проанализируйте полученные значения метрик, сделайте вывод о качестве обученной модели

In [ ]:
# your code here